In [ ]:
import kagglehub
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import shutil

# -----------------------------
# 1. Download dataset
# -----------------------------
path = kagglehub.dataset_download("saifulfadli/alpr-rbf")
print("Path to dataset files:", path)

# The dataset folder contains images inside subdirectories (class labels).
# We'll split into train and test manually.

base_dir = "alpr_dataset"
train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")

if not os.path.exists(base_dir):
    os.makedirs(train_dir)
    os.makedirs(test_dir)

    # Loop through classes and split files
    for class_name in os.listdir(path):
        class_path = os.path.join(path, class_name)
        if os.path.isdir(class_path):
            files = os.listdir(class_path)
            train_files, test_files = train_test_split(files, test_size=0.2, random_state=42)

            os.makedirs(os.path.join(train_dir, class_name))
            os.makedirs(os.path.join(test_dir, class_name))

            for f in train_files:
                shutil.copy(os.path.join(class_path, f), os.path.join(train_dir, class_name, f))

            for f in test_files:
                shutil.copy(os.path.join(class_path, f), os.path.join(test_dir, class_name, f))

print("Train/Test split done ✅")

# -----------------------------
# 2. Preprocess images
# -----------------------------
img_height, img_width = 128, 128
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical"
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical"
)

# -----------------------------
# 3. CNN Model with ReLU
# -----------------------------
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation="relu", input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation="relu"),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation="relu"),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(train_data.num_classes, activation="softmax")
])

model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

model.summary()

# -----------------------------
# 4. Train model
# -----------------------------
history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=10
)

# -----------------------------
# 5. Evaluate model
# -----------------------------
loss, acc = model.evaluate(test_data)
print(f"Test Accuracy: {acc*100:.2f}%")


100%|██████████| 273M/273M [00:09<00:00, 28.9MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/saifulfadli/alpr-rbf/versions/1


IsADirectoryError: [Errno 21] Is a directory: '/root/.cache/kagglehub/datasets/saifulfadli/alpr-rbf/versions/1/Datasets/number_plate'

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

base_dir = "alpr_dataset"
train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")

if not os.path.exists(base_dir):
    os.makedirs(train_dir)
    os.makedirs(test_dir)

    # Loop through classes and split files
    for class_name in os.listdir(path):
        class_path = os.path.join(path, class_name)
        if os.path.isdir(class_path):
            files = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]  # ✅ only files
            if len(files) == 0:
                continue  # skip empty folders

            train_files, test_files = train_test_split(files, test_size=0.2, random_state=42)

            os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
            os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

            for f in train_files:
                shutil.copy(os.path.join(class_path, f), os.path.join(train_dir, class_name, f))

            for f in test_files:
                shutil.copy(os.path.join(class_path, f), os.path.join(test_dir, class_name, f))

print("✅ Train/Test split completed successfully")


✅ Train/Test split completed successfully


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

# -----------------------------
# 1. Preprocess images
# -----------------------------
img_height, img_width = 128, 128
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    "alpr_dataset/train",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical"
)

test_data = test_datagen.flow_from_directory(
    "alpr_dataset/test",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical"
)

# -----------------------------
# 2. CNN Model with ReLU
# -----------------------------
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation="relu", input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation="relu"),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation="relu"),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(train_data.num_classes, activation="softmax")
])

model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

model.summary()

# -----------------------------
# 3. Train model
# -----------------------------
history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=10
)

# -----------------------------
# 4. Evaluate model
# -----------------------------
loss, acc = model.evaluate(test_data)
print(f"✅ Test Accuracy: {acc*100:.2f}%")


Found 0 images belonging to 1 classes.
Found 0 images belonging to 1 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


ValueError: The PyDataset has length 0

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
import glob

base_dir = "alpr_dataset"
train_dir = os.path.join(base_dir, "train/plates")
test_dir = os.path.join(base_dir, "test/plates")

if not os.path.exists(base_dir):
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # 🔍 find all images recursively
    image_extensions = ("*.jpg", "*.jpeg", "*.png", "*.bmp")
    all_images = []
    for ext in image_extensions:
        all_images.extend(glob.glob(os.path.join(path, "**", ext), recursive=True))

    print(f"Found {len(all_images)} images total")

    # split train/test
    train_files, test_files = train_test_split(all_images, test_size=0.2, random_state=42)

    # copy into folders
    for f in train_files:
        shutil.copy(f, os.path.join(train_dir, os.path.basename(f)))

    for f in test_files:
        shutil.copy(f, os.path.join(test_dir, os.path.basename(f)))

print("✅ Train/Test split completed successfully")


✅ Train/Test split completed successfully


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

# -----------------------------
# 1. Preprocess images
# -----------------------------
img_height, img_width = 128, 128
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    "alpr_dataset/train",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="binary"   # ✅ binary classification
)

test_data = test_datagen.flow_from_directory(
    "alpr_dataset/test",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="binary"   # ✅ binary classification
)

# -----------------------------
# 2. CNN Model with ReLU
# -----------------------------
model = models.Sequential([
    layers.Input(shape=(img_height, img_width, 3)),   # cleaner input definition

    layers.Conv2D(32, (3,3), activation="relu"),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation="relu"),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation="relu"),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(1, activation="sigmoid")   # ✅ sigmoid for binary
])

model.compile(optimizer="adam",
              loss="binary_crossentropy",   # ✅ binary loss
              metrics=["accuracy"])

model.summary()

# -----------------------------
# 3. Train model
# -----------------------------
history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=10
)

# -----------------------------
# 4. Evaluate model
# -----------------------------
loss, acc = model.evaluate(test_data)
print(f"✅ Test Accuracy: {acc*100:.2f}%")


Found 0 images belonging to 1 classes.
Found 0 images belonging to 1 classes.


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

ValueError: The PyDataset has length 0

In [ ]:
!ls alpr_dataset/train/plates | head
!ls alpr_dataset/test/plates | head


ls: cannot access 'alpr_dataset/train/plates': No such file or directory
ls: cannot access 'alpr_dataset/test/plates': No such file or directory


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
import glob

# Define correct folders
base_dir = "alpr_dataset"
train_dir = os.path.join(base_dir, "train/plates")
test_dir = os.path.join(base_dir, "test/plates")

# Remove old structure if it exists
if os.path.exists(base_dir):
    shutil.rmtree(base_dir)

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# 🔍 Find all images recursively in the original dataset
image_extensions = ("*.jpg", "*.jpeg", "*.png", "*.bmp")
all_images = []
for ext in image_extensions:
    all_images.extend(glob.glob(os.path.join(path, "**", ext), recursive=True))

print(f"Found {len(all_images)} images total")

# Split into train and test
train_files, test_files = train_test_split(all_images, test_size=0.2, random_state=42)

# Copy into correct folders
for f in train_files:
    shutil.copy(f, os.path.join(train_dir, os.path.basename(f)))

for f in test_files:
    shutil.copy(f, os.path.join(test_dir, os.path.basename(f)))

print("✅ Train/Test split completed successfully")


Found 237460 images total
✅ Train/Test split completed successfully


In [ ]:
!ls alpr_dataset/train/plates | head
!ls alpr_dataset/test/plates | head


0_1000_augmented.jpg
0_1001_augmented.jpg
0_1002_augmented.jpg
0_1005_augmented.jpg
0_1006_augmented.jpg
0_1007_augmented.jpg
0_1008_augmented.jpg
0_1009_augmented.jpg
0_100_asbdiuahnalk.jpg
0_100_augmented.jpg
0_1003_augmented.jpg
0_1004_augmented.jpg
0_100_pred.jpg
0_1013_augmented.jpg
0_1017_augmented.jpg
0_101_asbdiuahnalk.jpg
0_101_brwah.jpg
0_1020_augmented.jpg
0_1028_augmented.jpg
0_1029_augmented.jpg


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_height, img_width = 128, 128
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    "alpr_dataset/train",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="binary"
)

test_data = test_datagen.flow_from_directory(
    "alpr_dataset/test",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="binary"
)


Found 141203 images belonging to 1 classes.
Found 43909 images belonging to 1 classes.


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# -----------------------------
# 1. Settings
# -----------------------------
img_height, img_width = 28, 28
batch_size = 128  # bigger batch for speed

# -----------------------------
# 2. Load dataset (single class)
# -----------------------------
train_data = tf.keras.utils.image_dataset_from_directory(
    "alpr_dataset/train",
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode=None  # ✅ no labels
)

test_data = tf.keras.utils.image_dataset_from_directory(
    "alpr_dataset/test",
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode=None
)

# Normalize pixel values
train_data = train_data.map(lambda x: x / 255.0)
test_data = test_data.map(lambda x: x / 255.0)

# Cache & prefetch for speed
AUTOTUNE = tf.data.AUTOTUNE
train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
test_data = test_data.cache().prefetch(buffer_size=AUTOTUNE)

# -----------------------------
# 3. Build CNN Encoder
# -----------------------------
model = models.Sequential([
    layers.Input(shape=(img_height, img_width, 3)),

    layers.Conv2D(32, (3,3), activation="relu"),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation="relu"),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation="relu"),
    layers.Flatten(),
    layers.Dense(128, activation="relu")
])

model.summary()

# -----------------------------
# 4. Example: extract features
# -----------------------------
for batch in train_data.take(1):
    features = model(batch)
    print("Feature vector shape:", features.shape)


Found 141203 files.
Found 43909 files.


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)              │ (None, 26, 26, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 3, 3, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 128)            │       147,584 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 240,832 (940.75 KB)

 Trainable params: 240,832 (940.75 KB)

 Non-trainable params: 0 (0.00 B)

Feature vector shape: (128, 128)


In [ ]:
import os
import shutil
from PIL import Image
import numpy as np

# Paths
train_plates_dir = "alpr_dataset/train/plates"
test_plates_dir = "alpr_dataset/test/plates"

train_no_plates_dir = "alpr_dataset/train/no_plates"
test_no_plates_dir = "alpr_dataset/test/no_plates"

os.makedirs(train_no_plates_dir, exist_ok=True)
os.makedirs(test_no_plates_dir, exist_ok=True)

# Number of dummy images
num_train_dummy = 1000
num_test_dummy = 200

# Generate blank images (all black) as "no_plates"
for i in range(num_train_dummy):
    img = Image.fromarray(np.zeros((28,28,3), dtype=np.uint8))
    img.save(os.path.join(train_no_plates_dir, f"dummy_{i}.jpg"))

for i in range(num_test_dummy):
    img = Image.fromarray(np.zeros((28,28,3), dtype=np.uint8))
    img.save(os.path.join(test_no_plates_dir, f"dummy_{i}.jpg"))

print("✅ Dummy `no_plates` images created")


✅ Dummy `no_plates` images created


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 128

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    "alpr_dataset/train",
    target_size=(28,28),
    batch_size=batch_size,
    class_mode="binary"
)

test_data = test_datagen.flow_from_directory(
    "alpr_dataset/test",
    target_size=(28,28),
    batch_size=batch_size,
    class_mode="binary"
)


Found 142203 images belonging to 2 classes.
Found 44109 images belonging to 2 classes.


In [ ]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Input(shape=(28,28,3)),

    layers.Conv2D(32, (3,3), activation="relu"),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation="relu"),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(1, activation="sigmoid")  # binary
])

model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=1
)

loss, acc = model.evaluate(test_data)
print(f"✅ Test Accuracy: {acc*100:.2f}%")


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_23 (Conv2D)              │ (None, 26, 26, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_22 (MaxPooling2D) │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_24 (Conv2D)              │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_23 (MaxPooling2D) │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_8 (Flatten)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 224,449 (876.75 KB)

 Trainable params: 224,449 (876.75 KB)

 Non-trainable params: 0 (0.00 B)

1111/1111 ━━━━━━━━━━━━━━━━━━━━ 182s 162ms/step - accuracy: 0.9908 - loss: 0.0123 - val_accuracy: 1.0000 - val_loss: 0.0028
345/345 ━━━━━━━━━━━━━━━━━━━━ 23s 65ms/step - accuracy: 1.0000 - loss: 0.0025
✅ Test Accuracy: 100.00%


In [ ]:
import numpy as np
from PIL import Image
import os

train_no_plates_dir = "alpr_dataset/train/no_plates"
test_no_plates_dir = "alpr_dataset/test/no_plates"

# Replace dummy images with random noise
for i in range(1000):
    img = np.random.randint(0, 255, (28,28,3), dtype=np.uint8)
    Image.fromarray(img).save(os.path.join(train_no_plates_dir, f"rand_{i}.jpg"))

for i in range(200):
    img = np.random.randint(0, 255, (28,28,3), dtype=np.uint8)
    Image.fromarray(img).save(os.path.join(test_no_plates_dir, f"rand_{i}.jpg"))


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=0.1,
    horizontal_flip=True,
)

test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
layers.Dense(64, activation="relu"),
layers.Dense(1, activation="sigmoid")


<Dense name=dense_18, built=False>

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# -----------------------------
# 1. Generate realistic "no_plates"
# -----------------------------
no_plate_dir = "alpr_dataset/train/no_plates"
os.makedirs(no_plate_dir, exist_ok=True)

for i in range(500):  # Generate 500 fake no_plate images
    noise_img = np.random.randint(0, 256, (28, 28, 3), dtype=np.uint8)
    cv2.imwrite(os.path.join(no_plate_dir, f"noise_{i}.jpg"), noise_img)

# -----------------------------
# 2. Parameters
# -----------------------------
img_height, img_width = 28, 28
batch_size = 64

# -----------------------------
# 3. Data Augmentation
# -----------------------------
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2  # 20% validation
)

train_data = train_datagen.flow_from_directory(
    "alpr_dataset/train",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="binary",
    subset="training",
    shuffle=True
)

val_data = train_datagen.flow_from_directory(
    "alpr_dataset/train",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="binary",
    subset="validation",
    shuffle=True
)

# -----------------------------
# 4. Build CNN Model
# -----------------------------
model = models.Sequential([
    layers.Conv2D(16, (3,3), activation="relu", input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(32, (3,3), activation="relu"),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dropout(0.3),  # Prevent overfitting
    layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

# -----------------------------
# 5. Train Model
# -----------------------------
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=1
)

# -----------------------------
# 6. Evaluate
# -----------------------------
loss, acc = model.evaluate(val_data)
print(f"✅ Validation Accuracy: {acc*100:.2f}%")


Found 114963 images belonging to 2 classes.
Found 28740 images belonging to 2 classes.


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_25 (Conv2D)              │ (None, 26, 26, 16)     │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_24 (MaxPooling2D) │ (None, 13, 13, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (None, 11, 11, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_25 (MaxPooling2D) │ (None, 5, 5, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_9 (Flatten)             │ (None, 800)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 64)             │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 56,417 (220.38 KB)

 Trainable params: 56,417 (220.38 KB)

 Non-trainable params: 0 (0.00 B)

1797/1797 ━━━━━━━━━━━━━━━━━━━━ 246s 136ms/step - accuracy: 0.9937 - loss: 0.0221 - val_accuracy: 1.0000 - val_loss: 4.3558e-04
450/450 ━━━━━━━━━━━━━━━━━━━━ 31s 70ms/step - accuracy: 0.9999 - loss: 7.7439e-04
✅ Validation Accuracy: 100.00%


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# 1. Image size
img_height, img_width = 28, 28

# 2. Dataset
train_data = tf.keras.utils.image_dataset_from_directory(
    "alpr_dataset/train",
    image_size=(img_height, img_width),
    batch_size=32,
    label_mode="binary"
)

val_data = tf.keras.utils.image_dataset_from_directory(
    "alpr_dataset/test",
    image_size=(img_height, img_width),
    batch_size=32,
    label_mode="binary"
)

# Normalize
normalization_layer = layers.Rescaling(1./255)
train_data = train_data.map(lambda x, y: (normalization_layer(x), y))
val_data = val_data.map(lambda x, y: (normalization_layer(x), y))

# 3. ANN Model
model = models.Sequential([
    layers.Flatten(input_shape=(28,28,3)),       # Flatten image → vector
    layers.Dense(512, activation="tanh"),        # try tanh instead of relu
    layers.Dropout(0.3),
    layers.Dense(256, activation="sigmoid"),     # sigmoid hidden layer
    layers.Dropout(0.3),
    layers.Dense(1, activation="sigmoid")        # sigmoid for binary output
])

# 4. Compile
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

# 5. Train
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=1
)


Found 143703 files belonging to 2 classes.
Found 44309 files belonging to 2 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4491/4491 ━━━━━━━━━━━━━━━━━━━━ 118s 26ms/step - accuracy: 0.9902 - loss: 0.0283 - val_accuracy: 0.9981 - val_loss: 0.0058


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# 1. Image size
img_height, img_width = 28, 28

# 2. Dataset
train_data = tf.keras.utils.image_dataset_from_directory(
    "alpr_dataset/train",
    image_size=(img_height, img_width),
    batch_size=32,
    label_mode="binary"
)

val_data = tf.keras.utils.image_dataset_from_directory(
    "alpr_dataset/test",
    image_size=(img_height, img_width),
    batch_size=32,
    label_mode="binary"
)

# Normalize
normalization_layer = layers.Rescaling(1./255)
train_data = train_data.map(lambda x, y: (normalization_layer(x), y))
val_data = val_data.map(lambda x, y: (normalization_layer(x), y))

# 3. ANN Model
model = models.Sequential([
    layers.Flatten(input_shape=(28,28,3)),       # Flatten image → vector
    layers.Dense(512, activation="tanh"),        # try tanh instead of relu
    layers.Dropout(0.3),
    layers.Dense(256, activation="sigmoid"),     # sigmoid hidden layer
    layers.Dropout(0.3),
    layers.Dense(1, activation="sigmoid")        # sigmoid for binary output
])

# 4. Compile
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

# 5. Train
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=1
)


Found 143703 files belonging to 2 classes.
Found 44309 files belonging to 2 classes.
4489/4491 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9923 - loss: 0.0214